In [1]:
from dotenv import load_dotenv

load_dotenv("./.env", override=True)

True

In [2]:
from pydantic_settings import BaseSettings

class LLMConfig(BaseSettings):
    gemini_api_key: str
    model_name: str
    temperature: float

class EmbeddingConfig(BaseSettings):
    embedding_model_name: str
    gemini_api_key: str
    

In [3]:
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from pydantic import SecretStr
from langchain_milvus import Milvus

class ChatModel(ChatGoogleGenerativeAI):
    def __init__(self, config: LLMConfig):
        super().__init__(
            model=config.model_name,
            temperature=config.temperature,
            api_key=config.gemini_api_key
        )

class EmbeddingModel(GoogleGenerativeAIEmbeddings):
    def __init__(self, config: EmbeddingConfig):
        super().__init__(
            google_api_key=SecretStr(config.gemini_api_key),
            model=config.embedding_model_name
        )

class VectorDB(Milvus):
    def __init__(self, embedding_model: EmbeddingModel):
        super().__init__(
            embedding_function=embedding_model,
            connection_args={"uri": "./customer_support_knowledge_milvus.db"}
        )


In [4]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [5]:
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from typing import Any, Annotated

class SupportState(BaseModel):
    file_path: str
    query: str = Field(default="")
    query_type: str = Field(default="faq")
    chunks: list = Field(default=[])
    result: str = Field(default="")
    retriever: Any = Field(default=None)


In [6]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [7]:
def file_reader(file_path: str) -> str:
    """irrespective of the file type, read the file and return the content as string"""
    import os

    _, file_extension = os.path.splitext(file_path)
    file_extension = file_extension.lower()

    if file_extension == ".txt":
        with open(file_path, "r") as file:
            return file.read()
    elif file_extension == ".pdf":
        from PyPDF2 import PdfReader

        reader = PdfReader(file_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
        return text
    elif file_extension in [".docx", ".doc"]:
        from docx import Document

        doc = Document(file_path)
        text = ""
        for para in doc.paragraphs:
            text += para.text + "\n"
        return text
    else:
        raise ValueError(f"Unsupported file type: {file_extension}")

In [8]:
def load_and_split(state: SupportState):

    content = file_reader(state.file_path)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, chunk_overlap=100
    )
    chunks = text_splitter.split_text(content)
    state.chunks = chunks

    embedding_model = EmbeddingModel(EmbeddingConfig())
    vector_db = VectorDB(embedding_model)
    state.retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

    return state

def classify_query_type(state: SupportState):

    system_prompt = SystemMessagePromptTemplate.from_template(
        """You are a helpful customer support assistant. 
        Classify the user's query into one of the following categories: 'faq', 'technical_issue', 'billing', 'general_inquiry', 'handover'."""
    )

    human_prompt = HumanMessagePromptTemplate.from_template(
        "User query: {query}\n\nRespond with only the category name."
    )

    prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

    llm = ChatModel(LLMConfig())

    lcel = (prompt 
            | llm
            )
    
    response = lcel.invoke({"query": state.query})

    state.query_type = response.content.strip().lower()

    return state

def faq_node(state: SupportState):

    system_prompt = SystemMessagePromptTemplate.from_template(
        """You are a helpful customer support assistant. 
        Use the provided context to answer the user's query. If the answer is not in the context, respond with 'I don't know'."""
    )

    human_prompt = HumanMessagePromptTemplate.from_template(
        "Context: {context}\n\nUser query: {query}\n\nProvide a detailed response."
    )

    prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

    llm = ChatModel(LLMConfig())
    lcel = (prompt 
            | llm
            )
    
    context = "\n".join(state.chunks)
    response = lcel.invoke({"context": context, "query": state.query})

    state.result = response.content

    return state

def technical_issue_node(state: SupportState):

    system_prompt = SystemMessagePromptTemplate.from_template(
        """You are a technical support assistant. 
        Use the provided context to answer the user's technical issue. If the answer is not in the context, respond with 'I don't know'."""
    )

    human_prompt = HumanMessagePromptTemplate.from_template(
        "Context: {context}\n\nUser query: {query}\n\nProvide a detailed technical response."
    )

    prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

    llm = ChatModel(LLMConfig())
    lcel = (prompt 
            | llm
            )
    
    context = "\n".join(state.chunks)
    response = lcel.invoke({"context": context, "query": state.query})

    state.result = response.content

    return state

def billing_node(state: SupportState):

    system_prompt = SystemMessagePromptTemplate.from_template(
        """You are a billing support assistant. 
        Use the provided context to answer the user's billing query. If the answer is not in the context, respond with 'I don't know'."""
    )

    human_prompt = HumanMessagePromptTemplate.from_template(
        "Context: {context}\n\nUser query: {query}\n\nProvide a detailed billing response."
    )

    prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

    llm = ChatModel(LLMConfig())
    lcel = (prompt 
            | llm
            )
    
    context = "\n".join(state.chunks)
    response = lcel.invoke({"context": context, "query": state.query})

    state.result = response.content

    return state

def general_inquiry_node(state: SupportState):

    system_prompt = SystemMessagePromptTemplate.from_template(
        """You are a general support assistant. 
        Use the provided context to answer the user's general inquiry. If the answer is not in the context, respond with 'I don't know'."""
    )

    human_prompt = HumanMessagePromptTemplate.from_template(
        "Context: {context}\n\nUser query: {query}\n\nProvide a detailed response."
    )

    prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

    llm = ChatModel(LLMConfig())
    lcel = (prompt 
            | llm
            )
    
    context = "\n".join(state.chunks)
    response = lcel.invoke({"context": context, "query": state.query})

    state.result = response.content

    return state

def handover_node(state: SupportState):
    state.result = "Handover to human agent."
    return state

def condition(state: SupportState):
    """Route based on query type"""
    
    if state.query_type == "faq":
        return "faq_node"
    elif state.query_type == "technical_issue":
        return "technical_issue_node"
    elif state.query_type == "billing":
        return "billing_node"
    elif state.query_type == "general_inquiry":
        return "general_inquiry_node"
    elif state.query_type == "handover":
        return "handover_node"
    else:
        return "handover_node"

In [16]:
from langgraph.graph import StateGraph, START, END

workflow = StateGraph(
    state_schema=SupportState,
)

In [17]:
workflow.add_node("read_and_split", load_and_split)
workflow.add_node("classify_query", classify_query_type)
workflow.add_node("faq", faq_node)
workflow.add_node("technical_issue", technical_issue_node)
workflow.add_node("billing", billing_node)
workflow.add_node("general_inquiry", general_inquiry_node)
workflow.add_node("handover", handover_node)

workflow.add_edge(START, "read_and_split")
workflow.add_edge("read_and_split", "classify_query")
workflow.add_conditional_edges(
    "classify_query",
    condition,
    {
        "faq_node": "faq",
        "technical_issue_node": "technical_issue",
        "billing_node": "billing",
        "general_inquiry_node": "general_inquiry",
        "handover_node": "handover",
    }
)
workflow.add_edge("faq", END)
workflow.add_edge("technical_issue", END)
workflow.add_edge("billing", END)
workflow.add_edge("general_inquiry", END)
workflow.add_edge("handover", END)

app = workflow.compile()

In [21]:
result = app.invoke(
    {
        "query": "What are KPIs, and list all related to cutomer support?, and let me know is KPI related to customer support?",
        "file_path": "./customer-support-framework.pdf"
    }
)

In [23]:
print(result["result"])

Based on the context provided, here is a detailed response to your query:

**What are KPIs?**

According to the document, operational metrics or KPIs (Key Performance Indicators) are quantifiable measurements that help organizations monitor day-to-day progress. They provide tangible data and play a crucial role in decision-making for resource allocation, product development, and continuous process improvement.

**Is KPI related to customer support?**

Yes, KPIs are directly related to customer support. The context states that metrics and KPIs provide invaluable insights into the overall efficiency of a customer support team and allow organizations to benchmark their performance against industry standards.

**List of customer support related metrics and KPIs mentioned in the text:**

The document divides metrics into two types: operational (KPIs) and quality-based.

**Operational Metrics or KPIs:**
*   Average first response time
*   Preferred communication channels
*   Ticket resolutio